In [1]:
import pandas as pd
import re

from sklearn.decomposition import PCA, TruncatedSVD 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import StratifiedKFold
# from sklearn.semi_supervised import LabelSpreading
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import nltk
nltk.download('wordnet')
# from sklearn.preprocessing import StandardScaler

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Daria\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
for i in pd.read_csv('task2_data.csv').values:
    print(i)

['лося опасно ставить, сегодня баба на открытии проткнула 258, кто был на лосях получил страдания'
 False 'баба' 'сексизм']
['На очень долгий срок это  space, на дистанцию в 5 лет определенно БАБА!'
 False 'баба' 'сексизм']
['Значит нужно срочно продать Алибаба? А он у меня в минусе!' False 'баба'
 'сексизм']
['с Яндексом вы точно все спрогнозировали, но если бы знали про тинькова, то не по 4900 рекомендовали сбрасывать. Я, кстати сбросил по 4900'
 False 'яндекс' 'компании']
['с чего ты это взял ?)  если я действительно не эксперт и что мне рекомендовать человеку? ? скупай все акции Тинькофф!? Я посоветовал, то чем сам пользуюсь и на что опираюсь )'
 False 'тиньк' 'компании']
['посмотрим что будет. Мне на твой шорт Яндекс страшно смотреть)))))'
 False 'яндекс' 'компании']
['Момо в тиньке не шортится' False 'тиньк' 'компании']
['Я правильно понимаю, что вот эта облигация от Тинькофф {$RU000A102796} из этой же серии? Тут тоже дополнительный доход зависит от каких-то условий. Или есть при

['Мне все больше нравится тинькофф ?' False 'тиньк' 'компании']
['тесла и падает также быстро)))' False 'тесла' 'компании']
['И переходите на канал!разработал новую схему по тесла' False 'тесла'
 'компании']
['яндекс деньги сбер купил)' False 'яндекс' 'компании']
['я на втб торгуюсь т.к Тинькоф не особо доверяю и комиссия просто конь'
 False 'тиньк' 'компании']
['Можно расчеты посмотреть на справедливую цену Тинькофф в 1190? P S: моя справедливая оценка Тинькофф в 5000 руб'
 False 'тиньк' 'компании']
['поправляй своё бывает и разгоняй тучи по поводу Мостотреста. Нас всех поимели. Ротенберги как всегда в шоколаде'
 False ' рот' 'другое']
['Нууу, я тут вообще пару недель)) только прогу изучал(дрянь приличная) никому не говори, а то опять забанят??'
 False 'забан' 'другое']
['Что песня херня, что исполнитель' True 'пес' 'другое']
['ну не) судя по наждаку она уже об дно вкладки прогнозов стучаться должна))) такое ощущение что наждак только Тесла и держит))) ну и яблочко чуть чуть'
 False '

['Чушь не пори это тесла , ей пофиг на всё' False 'тесла' 'компании']
['это и есть поддержка тинькофф' False 'тиньк' 'компании']
['Да но тесла меньше многих упала и лучше восстанавливается' False 'тесла'
 'компании']
['тебя забанить надо потому, что правильно писать «ниггер»' False 'забан'
 'другое']
['та же песня было на 3275 пролетел теперь сижу в лонге выбора нет , придётся ждать'
 False 'пес' 'другое']
['да, да, конечно, в 17.30 и всё полетит к чертям, амеры веселей всегда были.'
 False 'черт' 'другое']
['которые от самого Тинькофф?' False 'тиньк' 'компании']
['Если меня читает ребята из Тинькофф или приближённые к ним , донести это до них ?'
 False 'тиньк' 'компании']
['А чем яндекс тебе не такой? Почему негатив?' False 'яндекс' 'компании']
['Байден вырвался вперёд' False 'байден' 'политика']
['у меня есть подозрение, что Трамп не такой жадный, как наш, и с таких дивов они просто забили брать налог)'
 True 'трамп' 'политика']
['Пелоси тоже хочет манипулировать рынком как трамп ))'

 False 'баба' 'сексизм']
['Нет.завтра откатится.она за сегодня два раза откатывала уже.не сильно но все же.я каждый раз думаю все.падает.а она опять)неугомонная @c17c3e4c-2d3b-463c-a933-be0fdc216674'
 False 'гомо' 'сексизм']
['тесла, да один из примеров, хайп  на безумных идеях и имени одного из хозяев.'
 False 'тесла' 'компании']
['Какой-то чёрный понедельник , хотя с Тинькофф каждый будний чёрный'
 False 'тиньк' 'компании']
['Эта баба давно только сосет тиньковскую сиську и attention whoreит. Чего вы от нее хотите...'
 True 'тиньк' 'компании']
['По онлайн подсчетам пока Байден лидирует' False 'байден' 'политика']
['Понял. Ухожу. Если байден и электричество не связаны... ???' False
 'байден' 'политика']
['Маск очень хорошо дружит , с Трампом' False 'трамп' 'политика']
['Полностью согласен, судя по отчёту самая перегретая бумага, да ещё Байден прибыль порежет и антимонопольное расследование возбудит.. дороже 90 лучше и не брать, а лучше 80 и ниже'
 False 'байден' 'политика']
['Трамп - 

In [3]:
pd.read_csv('entity_group.csv')

,word,group
0,гомо,сексизм
1,баба,сексизм
2,петух,сексизм
3,путин,политика
4,байден,политика
5,трамп,политика
6,навальн,политика
7,тиньк,компании
8,яндекс,компании
9,тесла,компании
